# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [4]:
def calculate_mt(leptons, photons, jets, met):
    # Particles
    if len(leptons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    
    # Transverse mass and Delta
    cos_delta_phi = np.cos(l.phi() - met.phi())
    mt = (2 * l.pt * met.pt * (1. - cos_delta_phi))**0.5
    
    return mt

In [13]:
def calculate_phi(leptons, photons, jets, met, eta_solution=0):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = calculate_mt(leptons, photons, jets, met)
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Select one of them
        if eta_solution > 0:
            eta_v = eta_v_plus
        elif eta_solution < 0:
            eta_v = eta_v_minus
        else:
            dice = np.random.rand()
            if dice > 0.5:
                eta_v = eta_v_plus
            else:
                eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = w + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector)
    l_ = l.boost(vv.boostvector)
    a_ = a.boost(vv.boostvector)
    w_ = w.boost(vv.boostvector)
    r_ = vv # vv.boost(vv.boostvector)

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector.unit()
    x_ = (r_.vector - z_ * r_.vector.dot(z_)).unit()
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l_.vector.dot(x_)
    ly_ = l_.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    
    
def calculate_phi_minus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=-1)
    
    
def calculate_phi_plus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=1)

In [21]:
def setup_observables(delphesprocessor):
    delphesprocessor.reset_observables()
        
    # Default observables (four-momenta of a, l, j, and met)
    delphesprocessor.add_default_observables(
        n_leptons_max=1,
        n_photons_max=1,
        n_jets_max=1,
        include_charge=False,
        include_numbers=False,
        include_met=True
    )
    
    # Lepton flavour
    delphesprocessor.add_observable('pdgid_l1', 'l[0].pdgid', required=True)

    # Two-particle systems
    delphesprocessor.add_observable('m_la', '(l[0] + a[0]).m', required=True)
    delphesprocessor.add_observable('m_lmet', '(l[0] + met).m', required=True)
    delphesprocessor.add_observable('m_amet', '(a[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_la','(l[0] + a[0]).pt', required=True)
    delphesprocessor.add_observable('pt_lmet','(l[0] + met).pt', required=True)
    delphesprocessor.add_observable('pt_amet','(a[0] + met).pt', required=True)
    delphesprocessor.add_observable('deltaphi_la', 'l[0].phi() - a[0].phi()', required=True)
    delphesprocessor.add_observable('deltaphi_lmet', 'l[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaphi_amet', 'a[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaeta_la', 'l[0].eta - a[0].eta', required=True)

    # Three-particle system
    delphesprocessor.add_observable('m_almet', '(a[0] + l[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_almet', '(a[0] + l[0] + met).pt', required=True)
    
    # mT(W) and ressurrection phi
    delphesprocessor.add_observable_from_function('mt', calculate_mt, required=True)
    delphesprocessor.add_observable_from_function('phi_minus', calculate_phi_minus, required=True)
    delphesprocessor.add_observable_from_function('phi_plus', calculate_phi_plus, required=True)
    delphesprocessor.add_observable_from_function('phi', calculate_phi, required=True)


In [25]:
def setup_cuts(delphesprocessor, use_tight_cuts=False):
    delphesprocessor.reset_cuts()
    
    if use_tight_cuts:
        delphesprocessor.add_cut(
            'pt_a1 >= 300.'
        )
        delphesprocessor.add_cut(
            'pt_l1 >= 80.'
        )
        delphesprocessor.add_cut(
            'et_miss >= 80.'
        )
        delphesprocessor.add_cut(
            '(deltaphi_la**2 + deltaeta_la**2)**0.5 >= 3.'
        )
        delphesprocessor.add_cut(
            'eta_l1**2 < 2.4**2'
        )
        
    else:
        delphesprocessor.add_cut(
            'pt_a1 >= 20.'
        )
        delphesprocessor.add_cut(
            'pt_l1 >= 20.'
        )
        delphesprocessor.add_cut(
            'et_miss >= 20.'
        )

## Main loop

In [26]:
n_runs_per_benchmark = 5  # Number of run_cards

In [27]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=True)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        dp.add_hepmc_sample(
            mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
            sampled_from_benchmark=benchmark
        )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_file=log_dir + '/delphes_{}.log'.format(run - 1),
        initial_command='source activate python2',
    )
    
    # Tight analysis
    setup_observables(dp)
    setup_cuts(dp, use_tight_cuts=True)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_tight_{}.h5'.format(i_card))
    
    # Loose analysis
    setup_observables(dp)
    setup_cuts(dp)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


10:11  Starting analysis of runs for card 0
10:11  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
10:11  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz
10:11  Calling gunzip -k /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz > None
10:12  Found weight labels in HEPMC file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
10:12  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz
10:12  Unzipping /Users/johannb

10:35    First 10 values for observable deltaphi_la:
[ 2.64580441  3.07845531         nan         nan -3.16945601 -2.82552046
 -3.10710722         nan  2.53683496  3.21447496]
10:35    First 10 values for observable deltaphi_lmet:
[-0.84387248  5.72389913 -6.1400013          nan -0.76269466  0.47517735
  0.14389342 -0.91495037 -2.40217781  1.21676199]
10:35    First 10 values for observable deltaphi_amet:
[-3.48967689  2.64544382         nan  2.36762899  2.40676135  3.3006978
  3.25100064         nan -4.93901277 -1.99771297]
10:36    First 10 values for observable deltaeta_la:
[ 2.07247871  0.99247307         nan         nan -1.15940538  0.44137256
 -0.8197751          nan -1.06639719  0.63949472]
10:36    First 10 values for observable m_almet:
[888.69792804 666.08791153          nan          nan 623.03519657
 719.91429912 745.71466189          nan 891.69647232 579.72670538]
10:36    First 10 values for observable pt_almet:
[120.97009012   4.17231831          nan          nan  17.7606

10:41    First 10 values for observable eta_a1:
[-0.69745928 -1.49736381  0.         -0.83585507  0.92265475 -0.34305939
  1.22818804  0.          1.24159205 -1.23796463]
10:42    First 10 values for observable phi_a1:
[-2.55713749  0.05831327  0.          1.6196543   2.10512924  1.88497329
  2.13904333  0.         -3.13382316 -3.13274527]
10:42    First 10 values for observable e_j1:
[ 232.4663405     0.         2718.53007438 1249.41557595    0.
    0.            0.          736.08513195  276.74347326    0.        ]
10:42    First 10 values for observable pt_j1:
[114.59998322   0.         323.66873169 234.98982239   0.
   0.           0.         249.18518066 213.63967896   0.        ]
10:42    First 10 values for observable eta_j1:
[ 1.33062541  0.          2.8177104  -2.35505438  0.          0.
  0.         -1.74627173 -0.74736351  0.        ]
10:42    First 10 values for observable phi_j1:
[ 2.44569135  0.          0.00426147 -1.68301582  0.          0.
  0.          1.08352435  1.4

10:51  Adding optional observable pt_l1 = l[0].pt with default 0.0
10:51  Adding optional observable eta_l1 = l[0].eta with default 0.0
10:51  Adding optional observable phi_l1 = l[0].phi() with default 0.0
10:51  Adding optional observable e_a1 = a[0].e with default 0.0
10:51  Adding optional observable pt_a1 = a[0].pt with default 0.0
10:51  Adding optional observable eta_a1 = a[0].eta with default 0.0
10:51  Adding optional observable phi_a1 = a[0].phi() with default 0.0
10:51  Adding optional observable e_j1 = j[0].e with default 0.0
10:51  Adding optional observable pt_j1 = j[0].pt with default 0.0
10:51  Adding optional observable eta_j1 = j[0].eta with default 0.0
10:51  Adding optional observable phi_j1 = j[0].phi() with default 0.0
10:51  Adding required observable pdgid_l1 = l[0].pdgid
10:51  Adding required observable m_la = (l[0] + a[0]).m
10:51  Adding required observable m_lmet = (l[0] + met).m
10:51  Adding required observable m_amet = (a[0] + met).m
10:51  Adding requir

10:56    73363 / 100000 events pass required observable pdgid_l1
10:56    62066 / 100000 events pass required observable m_la
10:56    73363 / 100000 events pass required observable m_lmet
10:56    86136 / 100000 events pass required observable m_amet
10:56    62066 / 100000 events pass required observable pt_la
10:56    73363 / 100000 events pass required observable pt_lmet
10:56    86136 / 100000 events pass required observable pt_amet
10:56    62066 / 100000 events pass required observable deltaphi_la
10:56    73363 / 100000 events pass required observable deltaphi_lmet
10:56    86136 / 100000 events pass required observable deltaphi_amet
10:56    62066 / 100000 events pass required observable deltaeta_la
10:56    62066 / 100000 events pass required observable m_almet
10:56    62066 / 100000 events pass required observable pt_almet
10:56    73363 / 100000 events pass required observable mt
10:56    62066 / 100000 events pass required observable phi_minus
10:56    62066 / 100000 even

...

11:34    First 10 values for observable eta_a1:
[ 0.21125469  1.11224937  0.         -0.80472451  0.89409524  2.19427681
 -1.60907328 -0.88329691 -1.9485122  -0.92999715]
11:34    First 10 values for observable phi_a1:
[ 2.97836471  0.30501842  0.         -0.8390314   1.20786321 -1.60168588
 -2.48184085 -2.07321668 -0.52718228 -0.24940291]
11:34    First 10 values for observable e_j1:
[131.81321398   0.           0.           0.         975.29251561
   0.           0.           0.           0.           0.        ]
11:34    First 10 values for observable pt_j1:
[23.4598484   0.          0.          0.         52.87163544  0.
  0.          0.          0.          0.        ]
11:35    First 10 values for observable eta_j1:
[2.41117215 0.         0.         0.         3.60721207 0.
 0.         0.         0.         0.        ]
11:35    First 10 values for observable phi_j1:
[-0.81431967  0.          0.          0.         -1.42235816  0.
  0.          0.          0.          0.        ]
1

11:39    First 10 values for observable et_miss:
[23.32974625 17.49790955 11.0487051  33.04911041 46.03778839 11.94730663
 36.95450592 23.27636147 14.53386879 45.87016678]
11:39    First 10 values for observable phi_miss:
[ 1.63785625  2.00949502  0.34817696  0.20988972  1.81128788  0.03975376
 -0.64445692 -2.82773685  3.10178018 -2.42651176]
11:39    First 10 values for observable e_visible:
[ 131.81321398  102.15045797  140.70573431  214.6753695  1107.40688843
   57.02935751  394.39428313  343.98991508    0.          164.24476521]
11:39    First 10 values for observable eta_visible:
[ 2.41117215e+00 -8.36032391e-01  1.84008532e+00 -1.34247541e+00
  2.93142370e+00 -6.60010278e-01  1.89518571e+00  1.73423731e+00
 -1.00000000e+11  1.45049798e+00]
11:39    First 10 values for observable e_l1:
[  0.          51.07522898  52.68850022 107.33768475  66.05718641
  28.51467875 197.19714156 171.99495754   0.          82.1223826 ]
11:39    First 10 values for observable pt_l1:
[ 0.         37.27

11:42  Calling gunzip -k /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_05/tag_1_pythia8_events.hepmc.gz > None
11:43  Found weight labels in HEPMC file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
11:43  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_05/tag_1_pythia8_events.hepmc.gz
11:43  Unzipping /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_05/tag_1_pythia8_events.hepmc.gz
11:43  Calling source activate python2; /Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes/DelphesHepMC /Users/johannbrehmer/work/projects/madminer/diboson_mining/cards/wgamma/delphes_card.dat /Users/johannbrehmer/work/projects/madminer/diboson_mining/data

11:50    First 10 values for observable deltaphi_lmet:
[ 0.17709041 -0.18861514  0.22759277         nan  0.11594224  0.48658037
 -0.44004583 -0.22537124  5.48636127 -0.14419681]
11:50    First 10 values for observable deltaphi_amet:
[-3.01579661  3.09664795  3.32288414 -3.17395014  3.65506589 -2.83243036
  3.08298165 -3.32398987         nan  3.24180073]
11:51    First 10 values for observable deltaeta_la:
[-1.89026515  0.91877963 -1.67937109         nan  0.27992259 -1.30157411
 -0.61265484  1.70261765         nan -0.65994263]
11:51    First 10 values for observable m_almet:
[1515.96641857 1177.21111449 2069.60066423           nan 1116.46780999
 1369.45956476 1057.93545632 1459.55530019           nan 1653.08428127]
11:51    First 10 values for observable pt_almet:
[ 72.39166578  23.34637267  63.1521408           nan 235.42842372
  85.37645065   6.43135483   5.13557283          nan 156.49615615]
11:52    First 10 values for observable mt:
[ 50.44675485  48.54989374  89.72796569          

11:59    First 10 values for observable phi_a1:
[-0.05420775  2.83584142  2.65294123 -2.56673861  1.20368326 -1.44787931
  0.58802778 -1.51197028  0.          2.47295475]
11:59    First 10 values for observable e_j1:
[  64.76605293  372.72421666   96.16153847  515.19214163  214.95084601
   97.63578684    0.            0.         1883.18867246  820.81949807]
11:59    First 10 values for observable pt_j1:
[ 44.49092865  22.58030701  64.34029388 485.47839355 161.49084473
  93.244133     0.           0.         690.65991211 145.90847778]
11:59    First 10 values for observable eta_j1:
[ 0.89685196 -3.49591708  0.94459158  0.30225223 -0.79121304  0.25591302
  0.          0.         -1.66075969 -2.41227984]
11:59    First 10 values for observable phi_j1:
[-0.08649581  1.12792838  0.78914648  0.16312897 -0.67878962  1.33357704
  0.          0.          0.01374887  1.71070683]
11:59    First 10 values for observable pdgid_l1:
[-13. -13. -11.  nan -11.  11. -11. -11. -13. -11.]
11:59    First 1

## Combine samples

In [28]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

12:02  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
12:02  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
12:02  Loading samples from file 1 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5
12:02  Loading samples from file 2 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_1.h5
12:02  Loading samples from file 3 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_2.h5
12:02  Loading samples from file 4 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples

In [29]:
filenames_in = [sample_dir + 'samples_tight_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples_tight.h5')

12:02  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
12:02  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight.h5
12:02  Loading samples from file 1 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_0.h5
12:02  Loading samples from file 2 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_1.h5
12:02  Loading samples from file 3 / 5 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_2.h5
12:02  Loading samples from file 4 / 5 at /Users/johannbrehmer/work/projects/madmin